In [1]:
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])

In [2]:
def alloc_param_pair(shape):
    weight = np.ramdom.normal(RND_MEAN, RND_STD, shape) #가중치 행렬
    bias = np.zeros(shape[-1]) #편향 벡터
    return {'w':weight, 'b':bias}

### 은닉 계층 하나를 위한 순전파 함수 정의

In [3]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, [x, hidden] #역전파 처리 과정에서 가중치 편미분 정보로서 hidden이 필요하기 때문에 전달

### 은닉 걔층 하나를 위한 역전파 함수 정의

In [4]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux
    
    g_output_w_out = hidden.transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose() # 출력 계층과 은닉 계층의 역전파 처리를 매개하는 과정
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis = 0)
    
    pm_hidden['w'] -= LENRAING_RATE * G_w_hid
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid
    
def relu_derv(y):
    return np.sign(y)

### 가변적 은닉 걔층 구성을 위한 파라미터 생성 함수 정의

In [5]:
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pn_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pn_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
        
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

### 가변적 은닉 계층 구성을 위한 순전파 함수 정의

In [6]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

### 가변적 은닉 계층 구성을 위한 역전파 함수 정의

In [8]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    
    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis = 0)
        
        g_hidden_hidden = pm.hiddens[n]['w'].transpose() #은닉층과 은닉층 사이의 매개
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hidden[n]['w'] -= LENRAING_RATE * G_w_hid
        pm_hidden[n]['b'] -= LEARNING_RATE * G_b_hid